# Integração Python com Outlook
### Funciona para qualquer e-mail: corporativo, gmail, hotmail, etc.

* Passo 1: Importar win32 e inicializar o outlook.

In [ ]:
import win32com.client as win32

outlook = win32.Dispatch('Outlook.Application')

* Passo 2: Escrever o e-mail e disparar.

In [ ]:
mail = outlook.CreateItem(0)
mail.To = "teste_email@example.com"
mail.CC = "teste_email2@example.com"
mail.BCC = "teste_email3@example.com"
mail.Subject = "Teste de e-mail com Python"
mail.Body = "Este é um teste de e-mail enviado via Python."
# ou mail.HTMLBody = "<html><body>Este é um teste de e-mail enviado com Python.</body></html>"

# Anexos (pode colocar quantos quiser):
attachment = r'C:\Users\nome_do_arquivo.pdf'
mail.Attachments.Add(attachment)

mail.Send()

## Desafio Python e E-mail

### Descrição

Digamos que você trabalha em uma indústria e está responsável pela área de inteligência de negócio.

Todo dia, você, a equipe ou até mesmo um programa, gera um report diferente para cada área da empresa:
- Financeiro
- Logística
- Manutenção
- Marketing
- Operaçções
- Produção
- Vendas

Cada um desses reports deve ser enviado por e-mail para o Gerente de cada área.

Crie um programa que façaa isso automaticamente. A relação de Gerentes (com seus respectivos e-mails) e áreas está no arquivo 'Enviar E-mails.xlsx'.

Dica: Use o pandas read_excel para ler o arquivo dos e-mails que isso vai facilitar.

In [ ]:
import pandas as pd
import win32com.client as win32
outlook = win32.Dispatch('outlook.application')
gerentes_df = pd.read_excel('Enviar E-mails.xlsx')
#gerentes_df.info()

for i, email in enumerate(gerentes_df['E-mail']):
    gerente = gerentes_df.loc[i, 'Gerente']
    area = gerentes_df.loc[i, 'Relatório']

    mail = outlook.CreateItem(0)
    mail.To = email
    mail.Subject = 'Relatório de {}'.format(area)
    mail.Body = '''
    Prezado {}, 
    Segue em anexo o Relatório de {}, conforme solicitado.
    Qualquer dúvida estou à disposição.
    Att.,
    '''.format(gerente, area)
    attachment  = r'C:\Users\joaop\Google Drive\Python Impressionador\{}.xlsx'.format(area)
    mail.Attachments.Add(attachment)

    mail.Send()

## Localizar Email e Anexo com Python

Podemos utilizar a biblioteca `imap_tools` para conectar ao servidor de e-mail e ler os e-mails. Iremos especificar o e-mail e senha para o programa (podendo usar um arquivo .env se for o caso) e depois a comunicação com o servidor de e-mail através de do `MailBox` usando o método `login`.

In [ ]:
from imap_tools import MailBox, AND # type: ignore
# from senha_email import minha_senha

usuario = "email_teste@gmail.com"
senha = 'Senha'

meu_email = MailBox("imap.gmail.com").login(usuario, senha)

Após isso podemos percorrer os e-mails com o método `fetch` e buscar pelos e-mails desejados através do e-mail do remetente e e-mail do destinatário e iterando sobre eles com um loop for para poder verificar se o assunto ou o corpo do e-mail ou titulo para constatar o conteúdo do e-mail. Dessa forma, podemos só "ler" os e-mails desejados mesmo. 

In [ ]:
# pegar email que foram enviados por um remetente específico

lista_emails = meu_email.fetch(AND(from_="email_teste@gmail.com", to="email_teste@gmail.com"))
for email in lista_emails:
    print(email.subject)
    print(email.text)

E para pegarmos os anexos de um e-mail caso haja algum, podemos receber a lista dos e-mails do remetente e iterar sobre eles com um loop for para poder verificar se há algum anexo nos e-mail. Se houve, podemos realizar mais uma iteração e pegar os valores do anexo desejado como uma carga útil (payload) usando o método `payload`. Dessa forma, podemos criar um arquivo em nossa máquina com exato o mesmo valor do anexo do e-mail. Ou até mesmo avaliar esse valor binário recebido do e-mail.

In [ ]:
# pegar anexo de um email
lista_emails = meu_email.fetch(AND(from_="email_teste@gmail.com"))
for email in lista_emails:
    if len(email.attachments) > 0:
        for anexo in email.attachments:
            if "ExemploAnexo" in anexo.filename:
                informacoes_anexo = anexo.payload
                with open("ExemploAnexo.xlsx", "wb") as arquivo_excel:
                    arquivo_excel.write(informacoes_anexo)

---

## Enviar e-mails pelo gmail com Python
Recentemente a Google adotou uma nova política para a autenticação do gmail. Dessa forma, mudou um pouco a forma de habilitar o envio de e-mails e conexões com o mesmo.
Podemos utilizar a biblioteca `smtplib` para conectar ao gmail e enviar os e-mails e utilizar a biblioteca `email.messagem` para escrever o corpo do e-mail mais corretamente.

In [ ]:
import smtplib
import email.message

def enviar_email():
    corpo_email = """
    <p>Parágrafo1</p>
    <p>Parágrafo2</p>
    """

    msg = email.message.Message()
    msg['Subject'] = "Assunto"
    msg['From'] = 'remetente'
    msg['To'] = 'destinatario'
    password = 'senha'
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(corpo_email )

    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()

    s.login(msg['From'], password)
    s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    print('Email enviado')

    enviar_email()

Dessa forma será possível sim enviar e-mails pelo gmail com Python, só que terá que criar uma senha de aplicativos no Gmail para que o programa possa usar ela.